In [2]:
import pandas as pd
import numpy as np


In [3]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [4]:
df1.pivot("도시", "연도", "인구")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [5]:
df1.set_index(["도시", "연도"])[["인구"]].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

In [6]:
df1.pivot(["지역", "도시"], "연도", "인구")

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

# groupby 메서드

## 그룹연산 메서드¶

groupby 결과, 즉 GroupBy 클래스 객체의 뒤에 붙일 수 있는 그룹연산 메서드는 다양하다. 다음은 자주 사용되는 그룹연산 메서드들이다.

- size, count: 그룹 데이터의 갯수
- mean, median, min, max: 그룹 데이터의 평균, 중앙값, 최소, 최대
- sum, prod, std, var, quantile : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수
- first, last: 그룹 데이터 중 가장 첫번째 데이터와 가장 나중 데이터
이 외에도 많이 사용되는 것으로는 다음과 같은 그룹연산이 있다.

- agg, aggregate
  - 만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 agg에 전달한다.
  - 또는 여러가지 그룹연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.
- describe
  - 하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.
- apply
  - describe 처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹연산이 없는 경우에 사용한다.
- transform
  - 그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.


예를 들어 다음과 같은 데이터가 있을 때 key1의 값(A 또는 B)에 따른 data1의 평균은 어떻게 구할까?


In [7]:
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [8]:
groups = df2.groupby(df2.key1)
groups

In [9]:
groups.groups

{'A': [0, 1, 4], 'B': [2, 3]}

In [10]:
groups.sum()

,data1,data2
key1,,
A,8,80
B,7,70


GroupBy 클래스 객체를 명시적으로 얻을 필요가 없다면 groupby 메서드와 그룹연산 메서드를 연속으로 호출한다. 다음 예제는 열 data1에 대해서만 그룹연산을 하는 코드이다

In [11]:
df2.data1.groupby(df2.key1).sum()

key1
A    8
B    7
Name: data1, dtype: int64

In [12]:
df2.groupby(df2.key1)["data1"].sum()   # `GroupBy` 클래스 객체에서 data1만 선택하여 분석하는 경우

key1
A    8
B    7
Name: data1, dtype: int64

In [13]:
df2.groupby(df2.key1).sum()["data1"]  # 전체 데이터를 분석한 후 data1만 선택한 경우

key1
A    8
B    7
Name: data1, dtype: int64

In [17]:
df2.groupby(df2.key2).sum()[['data2']]

,data2
key2,
one,90
two,60


# 연습 문제
key1의 값을 기준으로 data1의 값을 분류하여 합계를 구한 결과를 시리즈가 아닌 데이터프레임으로 구한다

In [14]:
df2.groupby(df2.key1).sum()[["data1"]] 

,data1
key1,
A,8
B,7


복합 키 (key1, key2) 값에 따른 data1의 합계

In [22]:
df2.data1.groupby([df2.key1, df2.key2]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [25]:
df2.data1.groupby([df2["key1"], df2["key2"]]).sum().unstack("key2")

key2,one,two
key1,,
A,6,2
B,3,4


In [26]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).sum().unstack("연도")

연도,2005,2010,2015
지역,,,
경상권,3512547,3393191,3448737
수도권,9762546,9894685,12794763


다음 데이터는 150 송이의 붓꽃(iris)에 대해 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width), 꽃잎폭(sepal_width), 꽃잎폭(sepal_width)을 측정한 데이터이다.

In [27]:
import seaborn as sns
iris = sns.load_dataset("iris")

In [29]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


각 붓꽃 종별로 가장 큰 값과 가장 작은 값의 비율을 구해보자. 이러한 계산을 하는 그룹연산 메서드는 없으므로 직접 만든 후 agg 메서드를 적용한다

In [37]:
iris.groupby(iris.species).min()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,4.3,2.3,1.0,0.1
versicolor,4.9,2.0,3.0,1.0
virginica,4.9,2.2,4.5,1.4


In [30]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()

iris.groupby(iris.species).agg(peak_to_peak_ratio)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


describe 메서드를 사용하면 다양한 기술 통계(descriptive statistics)값을 한 번에 구한다

하나의 데이터프레임이 생성된다는 점에 주의

In [38]:
iris.groupby(iris.species).describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

apply 메서드를 사용하면 describe 메서드처럼 하나의 그룹에 대해 하나의 대표값(스칼라 값)을 구하는 게 아니라 데이터프레임을 만들 수 있다

In [41]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3]

iris.groupby(iris.species).apply(top3_petal_length)

sepal_length  sepal_width  ...  petal_width     species
species                                    ...                         
setosa     24            4.8          3.4  ...          0.2      setosa
           44            5.1          3.8  ...          0.4      setosa
           23            5.1          3.3  ...          0.5      setosa
versicolor 83            6.0          2.7  ...          1.6  versicolor
           77            6.7          3.0  ...          1.7  versicolor
           72            6.3          2.5  ...          1.5  versicolor
virginica  118           7.7          2.6  ...          2.3   virginica
           117           7.7          3.8  ...          2.2   virginica
           122           7.7          2.8  ...          2.0   virginica

[9 rows x 5 columns]

transform 메서드는 그룹별 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터프레임 자체를 변화시킨다. 따라서 만들어진 데이터프레임의 크기는 원래 데이터프레임과 같다. 예를 들어 다음처럼 각 붓꽃 꽃잎길이가 해당 종 내에서 대/중/소 어느 것에 해당되는지에 대한 데이터프레임을 만들 수도 있다.

In [42]:
def q3cut(s):
    return pd.qcut(s, 3, labels=["소", "중", "대"]).astype(str)


iris["petal_length_class"] = iris.groupby(iris.species).petal_length.transform(q3cut)
iris[["petal_length", "petal_length_class"]].tail(10)

,petal_length,petal_length_class
140,5.6,중
141,5.1,소
142,5.1,소
143,5.9,대
144,5.7,중
145,5.2,소
146,5.0,소
147,5.2,소
148,5.4,중
149,5.1,소


# 연습 문제
붓꽃(iris) 데이터에서 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width) 등의 평균을 구하라. 만약 붓꽃 종(species)이 표시되지 않았을 때 이 수치들을 이용하여 붓꽃 종을 찾아낼 수 있을지 생각하라.

In [43]:
iris

,sepal_length,sepal_width,petal_length,petal_width,species,petal_length_class
0,5.1,3.5,1.4,0.2,setosa,소
1,4.9,3.0,1.4,0.2,setosa,소
2,4.7,3.2,1.3,0.2,setosa,소
3,4.6,3.1,1.5,0.2,setosa,중
4,5.0,3.6,1.4,0.2,setosa,소
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,소
146,6.3,2.5,5.0,1.9,virginica,소
147,6.5,3.0,5.2,2.0,virginica,소
148,6.2,3.4,5.4,2.3,virginica,중


In [48]:
iris.groupby(iris.species)['sepal_length', 'sepal_width'].describe()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


sepal_length                        ... sepal_width                 
                  count   mean       std  min  ...         25%  50%    75%  max
species                                        ...                             
setosa             50.0  5.006  0.352490  4.3  ...       3.200  3.4  3.675  4.4
versicolor         50.0  5.936  0.516171  4.9  ...       2.525  2.8  3.000  3.4
virginica          50.0  6.588  0.635880  4.9  ...       2.800  3.0  3.175  3.8

[3 rows x 16 columns]

In [50]:
iris.groupby(iris.species)['sepal_length','sepal_width'].mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,sepal_length,sepal_width
species,,
setosa,5.006,3.428
versicolor,5.936,2.770
virginica,6.588,2.974


# pivot_table
pivot_table
Pandas는 pivot 명령과 groupby 명령의 중간 성격을 가지는 pivot_table 명령도 제공한다.

pivot_table 명령은 groupby 명령처럼 그룹분석을 하지만 최종적으로는 pivot 명령처럼 피봇테이블을 만든다. 즉 groupby 명령의 결과에 unstack을 자동 적용하여 2차원적인 형태로 변형한다. 사용 방법은 다음과 같다.

- pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')
 - data: 분석할 데이터프레임 (메서드일 때는 필요하지 않음)
 - values: 분석할 데이터프레임에서 분석할 열
 - index: 행 인덱스로 들어갈 키 열 또는 키 열의 리스트
 - columns: 열 인덱스로 들어갈 키 열 또는 키 열의 리스트
 - aggfunc: 분석 메서드
 - fill_value: NaN 대체 값
 - margins: 모든 데이터를 분석한 결과를 오른쪽과 아래에 붙일지 여부
 - margins_name: 마진 열(행)의 이름


만약 조건에 따른 데이터가 유일하게 선택되지 않으면 그룹연산을 하며 이 때 aggfunc 인수로 정의된 함수를 수행하여 대표값을 계산한다.

pivot_table를 메서드로 사용할 때는 객체 자체가 데이터가 되므로 data 인수가 필요하지 않다.

예를 들어 위에서 만들었던 피봇테이블은 pivot_table 명령으로 다음과 같이 만들 수도 있다. 인수의 순서에 주의하라.

In [51]:
df1.pivot_table("인구", "도시", "연도")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [53]:
df1.pivot_table("인구", "도시", "연도",margins=True,margins_name='sum')

연도,2005,2010,2015,sum
도시,,,,
부산,3512547.0,3393191.0,3448737.0,3.451492e+06
서울,9762546.0,9631482.0,9904312.0,9.766113e+06
인천,NaN,263203.0,2890451.0,1.576827e+06
sum,6637546.5,4429292.0,5414500.0,5.350809e+06


In [54]:
df1["인구"].mean()

5350808.625

행 인덱스나 열 인덱스에 리스트를 넣으면 다중 인덱스 테이블을 만든다.

In [55]:
df1.pivot_table("인구", index=["연도", "도시"])

인구
연도   도시         
2005 부산  3512547
     서울  9762546
2010 부산  3393191
     서울  9631482
     인천   263203
2015 부산  3448737
     서울  9904312
     인천  2890451

In [56]:
df1.pivot_table("인구", ["도시", "연도"])

인구
도시 연도           
부산 2005  3512547
   2010  3393191
   2015  3448737
서울 2005  9762546
   2010  9631482
   2015  9904312
인천 2010   263203
   2015  2890451

식당에서 식사 후 내는 팁(tip)과 관련된 데이터를 이용하여 좀더 구체적으로 그룹분석 방법을 살펴본다. 우선 Seaborn 패키지에 설치된 샘플 데이터를 로드한다. 이 데이터프레임에서 각각의 컬럼은 다음을 뜻한다.

- total_bill: 식사대금
- tip: 팁
- sex: 성별
- smoker: 흡연/금연 여부
- day: 요일
- time: 시간
- size: 인원

In [57]:
tips = sns.load_dataset("tips")
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


식사대금와 팁의 비율을 나타내는 tip_pct를 추가

In [59]:
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [60]:
tips.describe()

,total_bill,tip,size,tip_pct
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


성별로 나누어 데이터 갯수를 세어본다

In [61]:
tips.groupby("sex").count()

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


size 명령을 사용하면 더 간단히 표시된다. size 명령은 NaN이 있어도 상관하지 않는다

In [62]:
tips.groupby('sex').size()

sex
Male      157
Female     87
dtype: int64

성별과 흡연유무로 나누어 데이터의 갯수를 알아본다

In [63]:
tips.groupby(['sex','smoker']).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

피봇 데이블 형태

In [71]:
tips.pivot_table('tip_pct','sex','smoker',aggfunc='count',margins=True)

smoker,Yes,No,All
sex,,,
Male,60,97,157
Female,33,54,87
All,93,151,244


성별과 흡연 여부에 따른 평균 팁 비율

In [65]:
tips.groupby('sex')[['tip_pct']].mean()

,tip_pct
sex,
Male,0.157651
Female,0.166491


In [66]:
tips.groupby('smoker')[['tip_pct']].mean()

,tip_pct
smoker,
Yes,0.163196
No,0.159328


pivot_table 명령을 사용할 수도 있다

In [67]:
tips.pivot_table('tip_pct',['sex','smoker'],aggfunc='mean')

tip_pct
sex    smoker          
Male   Yes     0.152771
       No      0.160669
Female Yes     0.182150
       No      0.156921

In [72]:
tips.pivot_table('tip_pct','sex','smoker',aggfunc='mean')

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


여성 혹은 흡연자의 팁 비율이 높은 것을 볼 수 있다. 하지만 이 데이터에는 평균을 제외한 분산(variance) 등의 다른 통계값이 없으므로 describe 명령으로 여러가지 통계값을 한 번에 알아본다

In [73]:
tips.groupby('sex')['tip_pct'].describe()

,count,mean,std,min,25%,50%,75%,max
sex,,,,,,,,
Male,157.0,0.157651,0.064778,0.035638,0.121389,0.153492,0.186240,0.710345
Female,87.0,0.166491,0.053632,0.056433,0.140416,0.155581,0.194266,0.416667


In [74]:
tips.groupby("smoker")[["tip_pct"]].describe()

tip_pct                      ...                              
         count      mean       std  ...       50%       75%       max
smoker                              ...                              
Yes       93.0  0.163196  0.085119  ...  0.153846  0.195059  0.710345
No       151.0  0.159328  0.039910  ...  0.155625  0.185014  0.291990

[2 rows x 8 columns]

In [75]:
tips.groupby(["sex", "smoker"])[["tip_pct"]].describe()

tip_pct                      ...                              
                count      mean       std  ...       50%       75%       max
sex    smoker                              ...                              
Male   Yes       60.0  0.152771  0.090588  ...  0.141015  0.191697  0.710345
       No        97.0  0.160669  0.041849  ...  0.157604  0.186220  0.291990
Female Yes       33.0  0.182150  0.071595  ...  0.173913  0.198216  0.416667
       No        54.0  0.156921  0.036421  ...  0.149691  0.181630  0.252672

[4 rows x 8 columns]

# 연습 문제 
- 팁의 비율이 요일과 점심/저녁 여부, 인원수에 어떤 영향을 받는지 살펴본다.
- 어떤 요인이 가장 크게 작용하는지 판단할 수 있는 방법이 있는가?

In [77]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [79]:
tips.groupby('day')['tip_pct'].describe()

,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
Thur,62.0,0.161276,0.038652,0.072961,0.138210,0.153846,0.192687,0.266312
Fri,19.0,0.169913,0.047665,0.103555,0.133739,0.155625,0.196637,0.263480
Sat,87.0,0.153152,0.051293,0.035638,0.123863,0.151832,0.188271,0.325733
Sun,76.0,0.166897,0.084739,0.059447,0.119982,0.161103,0.187889,0.710345


In [81]:
tips.groupby('time')['tip_pct'].describe()

,count,mean,std,min,25%,50%,75%,max
time,,,,,,,,
Lunch,68.0,0.164128,0.040242,0.072961,0.139147,0.154084,0.193917,0.266312
Dinner,176.0,0.159518,0.067477,0.035638,0.123192,0.155400,0.188209,0.710345


In [82]:
tips.groupby('size')['tip_pct'].describe()

,count,mean,std,min,25%,50%,75%,max
size,,,,,,,,
1,4.0,0.217292,0.080342,0.137931,0.170779,0.202752,0.249265,0.325733
2,156.0,0.165719,0.066848,0.035638,0.135223,0.156104,0.195036,0.710345
3,38.0,0.152157,0.045459,0.056433,0.124758,0.159323,0.186135,0.230742
4,37.0,0.145949,0.042395,0.077459,0.117750,0.146699,0.169797,0.280535
5,5.0,0.141495,0.067733,0.065660,0.106572,0.121389,0.172194,0.241663
6,4.0,0.156229,0.042153,0.103799,0.131654,0.162891,0.187466,0.195335


In [86]:
tips.pivot_table('tip_pct',['day','time'],'size',aggfunc='mean')

size                1         2         3         4         5         6
day  time                                                              
Thur Lunch   0.181728  0.164024  0.144599  0.145515  0.121389  0.173706
     Dinner       NaN  0.159744       NaN       NaN       NaN       NaN
Fri  Lunch   0.223776  0.181969  0.187735       NaN       NaN       NaN
     Dinner       NaN  0.162659       NaN  0.117750       NaN       NaN
Sat  Dinner  0.231832  0.155289  0.151439  0.138289  0.106572       NaN
Sun  Dinner       NaN  0.180870  0.152662  0.153168  0.159839  0.103799

각 그룹에서 가장 많은 팁과 가장 적은 팁의 차이를 알아보자

In [87]:
def peak_to_peak(x):
    return x.max() - x.min()


tips.groupby(["sex", "smoker"])[["tip"]].agg(peak_to_peak)

tip
sex    smoker      
Male   Yes     9.00
       No      7.75
Female Yes     5.50
       No      4.20

In [88]:
tips.groupby(["sex", "smoker"]).agg(["mean", peak_to_peak])[["total_bill"]]

total_bill             
                    mean peak_to_peak
sex    smoker                        
Male   Yes     22.284500        43.56
       No      19.791237        40.82
Female Yes     17.977879        41.23
       No      18.105185        28.58

In [89]:
tips.groupby(["sex", "smoker"]).agg(
    {'tip_pct': 'mean', 'total_bill': peak_to_peak})

tip_pct  total_bill
sex    smoker                      
Male   Yes     0.152771       43.56
       No      0.160669       40.82
Female Yes     0.182150       41.23
       No      0.156921       28.58

In [90]:
tips.pivot_table(['tip_pct', 'size'], ['sex', 'day'], 'smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
sex    day                                         
Male   Thur  2.300000  2.500000  0.164417  0.165706
       Fri   2.125000  2.000000  0.144730  0.138005
       Sat   2.629630  2.656250  0.139067  0.162132
       Sun   2.600000  2.883721  0.173964  0.158291
Female Thur  2.428571  2.480000  0.163073  0.155971
       Fri   2.000000  2.500000  0.209129  0.165296
       Sat   2.200000  2.307692  0.163817  0.147993
       Sun   2.500000  3.071429  0.237075  0.165710

In [91]:
tips.pivot_table('size', ['time', 'sex', 'smoker'], 'day',
                 aggfunc='sum', fill_value=0)

day                   Thur  Fri  Sat  Sun
time   sex    smoker                     
Lunch  Male   Yes       23    5    0    0
              No        50    0    0    0
       Female Yes       17    6    0    0
              No        60    3    0    0
Dinner Male   Yes        0   12   71   39
              No         0    4   85  124
       Female Yes        0    8   33   10
              No         2    2   30   43

# 연습 문제 
타이타닉 승객 데이터를 이용하여 다음 분석을 실시하라. 데이터는 다음과 같이 받을 수 있다.
```
titanic = sns.load_dataset("titanic")
```

- qcut 명령으로 세 개의 나이 그룹을 만든다.
- 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다. 생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다.
- 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.

In [94]:
titanic = sns.load_dataset("titanic")
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [97]:
label = ['child','young','old']

titanic['age_group']=pd.qcut(titanic.age,3,label)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_group
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,child
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,old
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,young
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,old
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,young
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,NaN
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,young


In [119]:
titanic_alive = titanic.loc[titanic['survived']==1]
titanic_alive

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_group
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,old
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,young
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,old
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False,young
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False,child
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,1,3,female,15.0,0,0,7.2250,C,Third,child,False,NaN,Cherbourg,yes,True,child
879,1,1,female,56.0,0,1,83.1583,C,First,woman,False,C,Cherbourg,yes,False,old
880,1,2,female,25.0,0,1,26.0000,S,Second,woman,False,NaN,Southampton,yes,False,young
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child


In [120]:
titanic.pivot_table('survived',['sex','age'],'pclass')

pclass          1    2     3
sex    age                  
female 0.75   NaN  NaN  1.00
       1.00   NaN  NaN  1.00
       2.00   0.0  1.0  0.25
       3.00   NaN  1.0  0.00
       4.00   NaN  1.0  1.00
...           ...  ...   ...
male   70.00  0.0  0.0   NaN
       70.50  NaN  NaN  0.00
       71.00  0.0  NaN   NaN
       74.00  NaN  NaN  0.00
       80.00  1.0  NaN   NaN

[145 rows x 3 columns]